In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Instructions for updating:
non-resource variables are not supported in the long term


word2Vec은 Word To Vector의 줄임말이다.  
딥러닝 모델은 입력값으로 수치값을 받기 때문에 자연어 처리 분야에서 단어(word)들을 어떻게 수치(vector)로 변환하느냐가 딥러닝 모델의 성능을 결정하는 중요한 요소이다.  

word2Vec의 장점은 의미가 비슷한 단어는 벡터 공간에서도 서로 가까운 위치에 존재하게 되는 비지도 학습으로 말뭉치 데이터만 있으면 손쉽게 word2Vec를 구현할 수 있다.  

데이터 획득

In [2]:
corpus = [
    'king is a strong man', 
    'queen is a wise woman', 
    'boy is a young man', 
    'girl is a young woman',
    'prince is a young king', 
    'princess is a young queen', 
    'man is strong', 
    'woman is pretty',
    'prince is a boy will be king', 
    'princess is a girl will be queen'
]
print(corpus)

['king is a strong man', 'queen is a wise woman', 'boy is a young man', 'girl is a young woman', 'prince is a young king', 'princess is a young queen', 'man is strong', 'woman is pretty', 'prince is a boy will be king', 'princess is a girl will be queen']


불용어(stop words) 제거하기  
불용어란? 학습에 도움이 되지도 않으면 빈번하게 발생되는 단어를 의미하며 효율적인 학습을 위해서 불용어를 데이터에서 제거한다.

In [3]:
# 불용어 제거 함수
def remove_stop_words(corpus):
    stop_words = ['is', 'a', 'will', 'be'] # 불용어 정의
    results = [] # 불용어를 제거한 결과를 저장해서 리턴시킬 빈 리스트
    # 불용어를 제거할 문장을 한줄씩 읽어가며 반복한다.
    for text in corpus:
        # print(text)
        # 불용어를 제거하기 위해서 공백을 경계로 문장을 단어 단위로 나눈다.
        temp = text.split()
        # print(temp)
        # 불용어 개수만큼 반복하며 불용어를 제거할 문장에서 불용어를 제거한다.
        for stop_word in stop_words:
            # 불용어를 제거한다.
            while stop_word in temp:
                temp.remove(stop_word)
            # ===== while
        # ===== for
        # print(temp)
        # 불용어가 제거된 단어 사이에 공백을 넣어서 불용어가 제거된 문장을 만든다.
        # print(' '.join(temp))
        results.append(' '.join(temp))
    return results

In [4]:
corpus = remove_stop_words(corpus)
corpus

['king strong man',
 'queen wise woman',
 'boy young man',
 'girl young woman',
 'prince young king',
 'princess young queen',
 'man strong',
 'woman pretty',
 'prince boy king',
 'princess girl queen']

In [5]:
# 불용어를 제거한 후 데이터에 존재하는 단어들은 다음과 같다.
words = []
for text in corpus:
    for word in text.split():
        words.append(word)
print(words) # 중복되는 단어가 존재한다.

['king', 'strong', 'man', 'queen', 'wise', 'woman', 'boy', 'young', 'man', 'girl', 'young', 'woman', 'prince', 'young', 'king', 'princess', 'young', 'queen', 'man', 'strong', 'woman', 'pretty', 'prince', 'boy', 'king', 'princess', 'girl', 'queen']


불용어를 제거한 단어의 중복을 제거한다.

In [6]:
words = set(words)
print(words) # 중복되는 단어가 존재하지 않는다.

{'girl', 'king', 'woman', 'strong', 'pretty', 'queen', 'princess', 'boy', 'wise', 'man', 'young', 'prince'}


단어별 인덱스 매핑 테이블 만들기  
단어를 원-핫 인코딩으로 변환하기 위해서 먼저 인덱스로 인코딩 해야하고, 단어를 손쉽게 인덱싱하기 위해서 key는 단어이고, value는 인덱스인 딕셔너리를 만든다.

In [7]:
for index, word in enumerate(words):
    print(index, word)

0 girl
1 king
2 woman
3 strong
4 pretty
5 queen
6 princess
7 boy
8 wise
9 man
10 young
11 prince


In [8]:
word2int = {}
for index, word in enumerate(words):
    word2int[word] = index
print(word2int)

{'girl': 0, 'king': 1, 'woman': 2, 'strong': 3, 'pretty': 4, 'queen': 5, 'princess': 6, 'boy': 7, 'wise': 8, 'man': 9, 'young': 10, 'prince': 11}


<img src="./images/skipgram.png" align="left" width="1300"/>

<img src="./images/skipgram2.png" align="left" width="1300"/>

<img src="./images/skipgram3.png" align="left" width="1300"/>

<img src="./images/skipgram4.png" align="left" width="1300"/>

<img src="./images/skipgram5.png" align="left" width="1300"/>

<img src="./images/skipgram6.png" align="left" width="1300"/>

skip gram 방식으로 각 단어별 레이블을 생성한다.

In [12]:
sentences = []
for sentence in corpus:
    sentences.append(sentence.split())
print(sentences)

[['king', 'strong', 'man'], ['queen', 'wise', 'woman'], ['boy', 'young', 'man'], ['girl', 'young', 'woman'], ['prince', 'young', 'king'], ['princess', 'young', 'queen'], ['man', 'strong'], ['woman', 'pretty'], ['prince', 'boy', 'king'], ['princess', 'girl', 'queen']]


In [20]:
WINDOW_SIZE = 2
data = []
# 단어별 레이블을 생성할 문장의 개수만큼 반복하며 레이블을 생성한다.
for sentence in sentences[:1]:
    #print('sentence: {}'.format(sentence))
    for index, word in enumerate(sentence):
        #print('index: {}, word: {}'.format(index, word))
        for neighbor in sentence[max(index - WINDOW_SIZE, 0):min(index + WINDOW_SIZE, len(sentence)) + 1]:
            if word != neighbor:
                #print('{} neighbor: {}'.format(word, neighbor))
                data.append([word, neighbor])
            # ===== if
        # ===== for neighbor
        #print('=' * 80)
    # ===== for index, word
# ===== for sentence

In [21]:
df = pd.DataFrame(data, columns=['input', 'label'])
df

,input,label
0,king,strong
1,king,man
2,strong,king
3,strong,man
4,man,king
5,man,strong


word2Vec 모델 구현하기